# 3.4 Actividad extra: Paralelización de un análisis de expresión génica

En este ejercicio se plantea un problema inspirado en el análisis de datos de expresión génica, una tarea habitual en Bioinformática y Biología Computacional. En este tipo de experimentos (por ejemplo, RNA-seq o microarrays), los datos se representan mediante una matriz en la que cada fila corresponde a un gen y cada columna a una muestra biológica.

Se simula un experimento de expresión génica generando una matriz de tamaño N×M, donde N representa el número de genes y M el número de muestras. Los valores de la matriz representan niveles de expresión génica simulados mediante una distribución aleatoria. A partir de estos datos, se plantea el siguiente análisis:

- Calcular la expresión media de cada gen.

- Determinar cuántos genes presentan una expresión media superior a un umbral dado, interpretándose estos como genes sobreexpresados.

El objetivo principal de esta actividad es comparar el rendimiento de distintas estrategias de implementación y paralelización en Python para resolver este problema, evaluando su eficiencia computacional y su escalado con el número de núcleos disponibles.

Concretamente, se implementan y comparan los siguientes métodos:

- Una versión secuencial en Python puro.

- Una versión vectorizada utilizando NumPy.

- Una versión paralela utilizando el módulo multiprocessing.

- Una versión compilada con Numba en modo secuencial.

- Una versión compilada con Numba utilizando paralelización mediante prange.

Las ejecuciones se realizan en un entorno de computación de altas prestaciones (HPC), utilizando el gestor de colas SLURM, y se analizan los tiempos de ejecución para diferentes configuraciones de núcleos (1, 2, 4 y 8 cores). De este modo, se estudia el impacto de la paralelización en el rendimiento del código y se comparan las ventajas y limitaciones de cada enfoque.

In [ ]:
# Imports + argumentos

import sys
import time
import os
import numpy as np

# Argumentos desde SLURM
if len(sys.argv) > 1:
    N_genes = int(sys.argv[1])
else:
    N_genes = 200_000

if len(sys.argv) > 2:
    nproc = int(sys.argv[2])
else:
    nproc = 1

M_samples = 50
threshold = 12.0

print("Genes:", N_genes)
print("Muestras:", M_samples)
print("Cores solicitados:", nproc)
print("NUMBA_NUM_THREADS:", os.environ.get("NUMBA_NUM_THREADS"))


In [ ]:
# Datos biológicos simulados 

expr = np.random.exponential(scale=10, size=(N_genes, M_samples))

In [ ]:
# Python secuencial

def count_overexpressed_py(expr, threshold):
    count = 0
    for i in range(expr.shape[0]):
        s = 0.0
        for j in range(expr.shape[1]):
            s += expr[i, j]
        if s / expr.shape[1] > threshold:
            count += 1
    return count

In [ ]:
# Optimización con NumPy

def count_overexpressed_numpy(expr, threshold):
    means = np.mean(expr, axis=1)
    return np.sum(means > threshold)


In [ ]:
# Optimización con Numba secuencial y paralelo (prange)

from numba import njit, prange

@njit
def count_overexpressed_numba(expr, threshold):
    count = 0
    for i in range(expr.shape[0]):
        s = 0.0
        for j in range(expr.shape[1]):
            s += expr[i, j]
        if s / expr.shape[1] > threshold:
            count += 1
    return count

@njit(parallel=True)
def count_overexpressed_numba_par(expr, threshold):
    count = 0
    for i in prange(expr.shape[0]):
        s = 0.0
        for j in range(expr.shape[1]):
            s += expr[i, j]
        if s / expr.shape[1] > threshold:
            count += 1
    return count


In [ ]:
# Optimización con Multiprocessing 

from multiprocessing import Pool

def partial_count(args):
    block, threshold = args
    means = np.mean(block, axis=1)
    return np.sum(means > threshold)

def count_overexpressed_mp(expr, threshold, nproc):
    blocks = np.array_split(expr, nproc)
    with Pool(nproc) as p:
        counts = p.map(
            partial_count,
            [(block, threshold) for block in blocks]
        )
    return sum(counts)


In [ ]:
# Warm-up para Numba

expr_warm = expr[:1000, :]
_ = count_overexpressed_numba(expr_warm, threshold)
_ = count_overexpressed_numba_par(expr_warm, threshold)

In [ ]:
# Mediciones

# Código original (solo 1 core)
if nproc == 1:
    t0 = time.time()
    res_py = count_overexpressed_py(expr, threshold)
    t1 = time.time()
    time_py = t1 - t0
else:
    res_py = None
    time_py = None

# NumPy
t0 = time.time()
res_np = count_overexpressed_numpy(expr, threshold)
t1 = time.time()
time_np = t1 - t0

# Numba secuencial
t0 = time.time()
res_numba = count_overexpressed_numba(expr, threshold)
t1 = time.time()
time_numba = t1 - t0

# Numba paralelo
t0 = time.time()
res_numba_par = count_overexpressed_numba_par(expr, threshold)
t1 = time.time()
time_numba_par = t1 - t0

# Multiprocessing
t0 = time.time()
res_mp = count_overexpressed_mp(expr, threshold, nproc)
t1 = time.time()
time_mp = t1 - t0


In [ ]:
print("\nResultados (cores =", nproc, ")")
print("----------------------------------")

if time_py is not None:
    print("Python secuencial")
    print("  Genes sobreexpresados:", res_py)
    print("  Tiempo:", time_py, "s")

print("NumPy")
print("  Genes sobreexpresados:", res_np)
print("  Tiempo:", time_np, "s")

print("Numba secuencial")
print("  Genes sobreexpresados:", res_numba)
print("  Tiempo:", time_numba, "s")

print("Numba paralelo")
print("  Genes sobreexpresados:", res_numba_par)
print("  Tiempo:", time_numba_par, "s")

print("Multiprocessing")
print("  Genes sobreexpresados:", res_mp)
print("  Tiempo:", time_mp, "s")

## Resultados

**Número de genes:** 200 000
**Número de muestras:** 50

---

**Número de núcleos: 1**

* Python secuencial:
  Resultado: 16 886 genes sobreexpresados
  Tiempo: 2.16 s

* NumPy:
  Resultado: 16 886 genes sobreexpresados
  Tiempo: 0.0107 s

* Numba secuencial:
  Resultado: 16 886 genes sobreexpresados
  Tiempo: 0.0081 s

* Numba `@njit(parallel=True)`:
  Resultado: 16 886 genes sobreexpresados
  Tiempo: 0.0081 s

* Multiprocessing:
  Resultado: 16 886 genes sobreexpresados
  Tiempo: 0.199 s

---

**Número de núcleos: 2**

* NumPy:
  Resultado: 16 904 genes sobreexpresados
  Tiempo: 0.0098 s

* Numba secuencial:
  Resultado: 16 904 genes sobreexpresados
  Tiempo: 0.0082 s

* Numba `@njit(parallel=True)`:
  Resultado: 16 904 genes sobreexpresados
  Tiempo: 0.0046 s

* Multiprocessing:
  Resultado: 16 904 genes sobreexpresados
  Tiempo: 0.155 s

---

**Número de núcleos: 4**

* NumPy:
  Resultado: 16 631 genes sobreexpresados
  Tiempo: 0.0135 s

* Numba secuencial:
  Resultado: 16 631 genes sobreexpresados
  Tiempo: 0.0080 s

* Numba `@njit(parallel=True)`:
  Resultado: 16 631 genes sobreexpresados
  Tiempo: 0.0026 s

* Multiprocessing:
  Resultado: 16 631 genes sobreexpresados
  Tiempo: 0.144 s

---

**Número de núcleos: 8**

* NumPy:
  Resultado: 16 823 genes sobreexpresados
  Tiempo: 0.0098 s

* Numba secuencial:
  Resultado: 16 823 genes sobreexpresados
  Tiempo: 0.0081 s

* Numba `@njit(parallel=True)`:
  Resultado: 16 823 genes sobreexpresados
  Tiempo: 0.0021 s

* Multiprocessing:
  Resultado: 16 823 genes sobreexpresados
  Tiempo: 0.140 s

---

## Interpretación y comparación de resultados

El número de genes sobreexpresados varía ligeramente entre ejecuciones debido a que los datos de expresión se generan de forma aleatoria en cada ejecución. No obstante, para una misma ejecución, todas las implementaciones devuelven exactamente el mismo resultado, lo que confirma la corrección de las versiones paralelas.

Los resultados obtenidos muestran un comportamiento coherente con las características de cada método y lo ya visto en las anteriores actividades de este laboratorio.

La versión secuecial en Python presenta el peor rendimiento, con un tiempo de ejecución claramente peor al resto. Este resultado es esperable, ya que el cálculo se realiza mediante bucles explícitos en Python sin ningún tipo de optimización ni paralelización. Por este motivo, esta versión se utiliza únicamente como referencia.

La implementación mediante NumPy es muy eficiente y presenta tiempos prácticamente constantes independientemente del número de núcleos. Para este tamaño de problema, NumPy ofrece un rendimiento excelente sin necesidad de paralelización explícita. 

La versión Numba secuencial mejora notablemente respecto al Python puro y alcanza tiempos similares a NumPy. Esto confirma que la compilación JIT de Numba elimina gran parte del overhead asociado a los bucles en Python, aunque no aprovecha múltiples núcleos.

La implementación Numba paralela usando prange es la que mejor escala con el número de núcleos. A medida que se incrementa el número de threads, el tiempo de ejecución disminuye de forma clara, alcanzando el mejor rendimiento con 8 cores. Esto indica que el problema es altamente paralelizable y que el paralelismo basado en hilos y memoria compartida resulta muy eficiente para este tipo de operación.

Por el contrario, la versión basada en multiprocessing presenta tiempos significativamente mayores y una mejora limitada al aumentar el número de procesos. Esto se debe al elevado overhead asociado a la creación de procesos y a la división y comunicación de grandes bloques de datos entre ellos, lo que penaliza el rendimiento en comparación con las soluciones basadas en memoria compartida.

En conjunto, estos resultados muestran que para este tipo de análisis de datos biológicos, las estrategias basadas en NumPy y Numba con paralelización mediante prange son las más adecuadas, mientras que el uso de multiprocessing resulta poco eficiente para operaciones de reducción sobre grandes arrays.